In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fashionmnist' dataset.
Path to dataset files: /kaggle/input/fashionmnist


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

torch.manual_seed(42)

base="/kaggle/input/fashionmnist"

In [3]:
device=torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [4]:
device

device(type='cuda')

In [5]:
df=pd.read_csv(f"{base}/fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [7]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0])

In [8]:
class CustomDataset(Dataset):
    def __init__(self,X,y):
        self.x=torch.tensor(X,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.long)
        
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self,index):
        return self.x[index,:]/255.0,self.y[index]

In [9]:
train_dataset=CustomDataset(X_train.values,y_train.values)
test_dataset=CustomDataset(X_test.values,y_test.values)

In [10]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.6196, 0.4980, 0.4235, 0.4392, 0.3843, 0.4196, 0.3922, 0.4588,
         0.4588, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0588, 1.0000, 0.8588, 0.9412, 0.9843, 0.9529, 0.9569, 0.8471,
         0.8902, 0.9961, 0.0353, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3176, 0.8902, 0.6745, 0.8275, 0.8392, 0.7647, 0.8078,
         0.8000, 0.7373, 0.9373, 0.2784, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.5882, 0.8824, 0.8314, 0.8627, 0.8510, 0.7725,
         0.8431, 0.8941, 0.8039, 0.9020, 0.5294, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [11]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=True,pin_memory=True)

In [12]:
class ann(nn.Module):
    def __init__(self,inp_features):
        super().__init__()
        self.model=nn.Sequential(
            nn.Linear(inp_features,128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(num_features=128),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(num_features=64),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(num_features=32),
            nn.Linear(32,10)
            )
        # nn.CrossEntropyLoss expects raw logits, not probabilities.
        
    def forward(self,x):
        return self.model(x)

In [13]:
epochs = 30
learning_rate = 0.1

In [14]:
# model
model=ann(X_train.shape[1])
model=model.to(device)

# loss fn
loss=nn.CrossEntropyLoss()

# optimizer
reg_coef=1e-4
optimizer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=reg_coef)

In [24]:
# training loop
for i in range(epochs):
    total_epoch_loss = 0
    correct=0
    total=0
    for x_batch,y_batch in train_loader:
        x_batch,y_batch =x_batch.to(device),y_batch.to(device)
         
        pred=model(x_batch)
        
        lss=loss(pred,y_batch)
        
        optimizer.zero_grad()
        lss.backward()
        
        optimizer.step()
        
        total+=y_batch.shape[0]
        
        # print(pred)
        correct+=(y_batch==pred.argmax(dim=1)).sum().item()
        
        total_epoch_loss = total_epoch_loss + lss.item()
        
    print(f'Epoch: {i + 1} , Loss: {total_epoch_loss/len(train_loader)} , Accuracy : {correct/total}')

Epoch: 1 , Loss: 0.9391295889971014 , Accuracy : 0.6664888888888889
Epoch: 2 , Loss: 0.83544686029969 , Accuracy : 0.7171333333333333
Epoch: 3 , Loss: 0.7832995739784661 , Accuracy : 0.7410666666666667
Epoch: 4 , Loss: 0.7633698946017277 , Accuracy : 0.7484666666666666
Epoch: 5 , Loss: 0.73654355745241 , Accuracy : 0.7586444444444445
Epoch: 6 , Loss: 0.7399241070037372 , Accuracy : 0.7563777777777778
Epoch: 7 , Loss: 0.7105879534006797 , Accuracy : 0.7695777777777778
Epoch: 8 , Loss: 0.7027596152667552 , Accuracy : 0.7713555555555556
Epoch: 9 , Loss: 0.6843495651412366 , Accuracy : 0.7783555555555556
Epoch: 10 , Loss: 0.6674943623512284 , Accuracy : 0.7829111111111111
Epoch: 11 , Loss: 0.6504337049983627 , Accuracy : 0.7885333333333333
Epoch: 12 , Loss: 0.6658185876127499 , Accuracy : 0.7855555555555556
Epoch: 13 , Loss: 0.6567322997099639 , Accuracy : 0.7878888888888889
Epoch: 14 , Loss: 0.652774627391294 , Accuracy : 0.7898888888888889
Epoch: 15 , Loss: 0.6469966593306371 , Accuracy 

In [25]:
# model to eval mode, This does not change weights. It only changes how certain layers behave.(like dropout)
model.eval()

ann(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.2, inplace=False)
    (11): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [26]:
# evaluation 
with torch.no_grad():
    total_loss = 0
    correct=0
    total=0
    for x_batch,y_batch in test_loader:
        x_batch,y_batch =x_batch.to(device),y_batch.to(device)
         
        pred=model(x_batch)
        
        lss=loss(pred,y_batch)
        total+=y_batch.shape[0]
        
        # print(pred)
        correct+=(y_batch==pred.argmax(dim=1)).sum().item()
        
        total_loss = total_loss + lss.item()
        
    print(f'Loss: {total_loss/len(test_loader)} , Accuracy : {correct/total}')

Loss: 2.3880157316925685 , Accuracy : 0.8552666666666666


# CNN

In [107]:
from torchvision import transforms

# Define data augmentations for the training dataset
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

In [116]:
# create CustomDataset Class
class CustomDataset(Dataset):

  def __init__(self, features, labels,transform):

    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)
    self.transform=transform
  def __len__(self):

    return len(self.features)

  def __getitem__(self, index):
    feature=self.features[index]
    label=self.labels[index]
    # x.squeeze(0)  # will remove dim-0 only if its size == 1
    feature = self.transform(feature.squeeze(0).numpy())
    return feature,label

In [117]:
# create dataset's  objects
train_dataset = CustomDataset(X_train.values, y_train.values,transform=train_transform)
test_dataset = CustomDataset(X_test.values, y_test.values,transform=test_transform)

train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=32, shuffle=True)

In [118]:
# define NN class

class MyNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Flatten(),
        nn.Linear(32*5*5, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10),
    )

  def forward(self, x):

    return self.model(x)

In [119]:
# set learning rate and epochs
epochs = 10
learning_rate = 0.01

In [120]:
# instatiate the model
model = MyNN(X_train.shape[1])

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [121]:
!pip install torchsummary

In [122]:
from torchsummary import summary
model=model.to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
         MaxPool2d-3           [-1, 16, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           4,640
              ReLU-5           [-1, 32, 11, 11]               0
         MaxPool2d-6             [-1, 32, 5, 5]               0
           Flatten-7                  [-1, 800]               0
            Linear-8                  [-1, 128]         102,528
              ReLU-9                  [-1, 128]               0
           Linear-10                   [-1, 64]           8,256
             ReLU-11                   [-1, 64]               0
           Linear-12                   [-1, 10]             650
Total params: 116,234
Trainable params: 116,234
Non-trainable params: 0
-------------------------------

In [123]:
# training loop
for i in range(epochs):
    total_epoch_loss = 0
    correct=0
    total=0
    for x_batch,y_batch in train_loader:
        x_batch,y_batch =x_batch.to(device),y_batch.to(device)
         
        pred=model(x_batch)
        
        lss=loss(pred,y_batch)
        
        optimizer.zero_grad()
        lss.backward()
        
        optimizer.step()
        
        total+=y_batch.shape[0]
        
        # print(pred)
        correct+=(y_batch==pred.argmax(dim=1)).sum().item()
        
        total_epoch_loss = total_epoch_loss + lss.item()
        
        
    print(f'Epoch: {i + 1} , Loss: {total_epoch_loss/len(train_loader)} , Accuracy : {correct/total}')

Epoch: 1 , Loss: 2.2214442624550976 , Accuracy : 0.19502222222222224
Epoch: 2 , Loss: 1.341269395414107 , Accuracy : 0.4758
Epoch: 3 , Loss: 0.9836116427848249 , Accuracy : 0.6279333333333333
Epoch: 4 , Loss: 0.8537195193149049 , Accuracy : 0.6735111111111111
Epoch: 5 , Loss: 0.7665797647806288 , Accuracy : 0.7061555555555555
Epoch: 6 , Loss: 0.7155481771471319 , Accuracy : 0.7266666666666667
Epoch: 7 , Loss: 0.6725521347225348 , Accuracy : 0.7443333333333333
Epoch: 8 , Loss: 0.6379386036846768 , Accuracy : 0.7571111111111111
Epoch: 9 , Loss: 0.6122631870882682 , Accuracy : 0.7681333333333333
Epoch: 10 , Loss: 0.594287302215301 , Accuracy : 0.7758222222222222


In [99]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=800, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): ReLU()
    (11): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [100]:
# evaluation 
with torch.no_grad():
    total_loss = 0
    correct=0
    total=0
    for x_batch,y_batch in test_loader:
        x_batch,y_batch =x_batch.to(device),y_batch.to(device)
         
        pred=model(x_batch)
        
        lss=loss(pred,y_batch)
        total+=y_batch.shape[0]
        
        # print(pred)
        correct+=(y_batch==pred.argmax(dim=1)).sum().item()
        
        total_loss = total_loss + lss.item()
        
    print(f'Loss: {total_loss/len(test_loader)} , Accuracy : {correct/total}')

Loss: 0.6546633135217593 , Accuracy : 0.7726666666666666
